<a href="https://colab.research.google.com/github/bryanjiang1204/aurora/blob/main/Together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Video**

In [ ]:
!pip install fvcore
!pip install av
!pip install pytorchvideo

In [ ]:
import torch

In [ ]:
model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/X3D_S.pyth" to /root/.cache/torch/hub/checkpoints/X3D_S.pyth
100%|██████████| 29.4M/29.4M [00:01<00:00, 30.1MB/s]


In [ ]:
import urllib

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import CenterCropVideo, NormalizeVideo
from pytorchvideo.transforms import ApplyTransformToKey,ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo

mean = [0.5, 0.5, 0.5]
std = [0.25, 0.25, 0.25]

num_frames = 13
sampling_rate = 6
frames_per_second = 30

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=182),
            CenterCropVideo(
                crop_size=(182, 182)
            )
        ]
    ),
)

clip_duration = (num_frames * sampling_rate)/frames_per_second

url_link = "https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4"
video_path = 'archery.mp4'
try: urllib.URLopener().retrieve(url_link, video_path)
except: urllib.request.urlretrieve(url_link, video_path)

video = EncodedVideo.from_path(video_path)

video_data = video.get_clip(start_sec=0, end_sec=3)
video_data = transform(video_data)

inputs = video_data["video"]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [ ]:
from torchvision.models.feature_extraction import create_feature_extractor

class Video_Embeddings(torch.nn.Module):
    def __init__(self):
        super().__init__()
        model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True).eval()
        self.body = create_feature_extractor(model, return_nodes={'blocks.5.pool.post_conv': 'embeddings',})

    def forward(self, x):
        x = self.body(x)
        return x

extractor = Video_Embeddings()
flat_layer = extractor(inputs[None,...])

y = torch.flatten(flat_layer['embeddings'])
y.shape

**Text**

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch

input_text = "Wise Gurus chant"

input_ids = torch.tensor([tokenizer.encode(input_text)])
with torch.no_grad():
    model_output = model(input_ids)
bert_embeddings = model_output[0]

x = torch.flatten(bert_embeddings)
x.shape
#padding

Concotanate

In [ ]:
z = torch.cat((x,y))
z.shape

torch.Size([12800])

Questions:

How to get Audio Embeddings.

If the number of words spoken is more than 50, the text embeddings length can go as high as 50,000 - 100,000. How to proportionalize. Or perhaps another solution.

What is best video model to use. If a video model's original dimensions are [2048,2,2] does flattening to make it [8192] count as an embedding